* 请在环境变量中设置`DB_URI`指向数据库

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from PyFin.api import *
from alphamind.api import *
from alphamind.strategy.strategy import Strategy, RunningSetting
from matplotlib import pyplot as plt

plt.style.use('ggplot')

In [ ]:
# Back test parameter settings
start_date = '2016-01-01'
end_date = '2018-03-30'

freq = '10b'
industry_name = 'sw'
industry_level = 1
turn_over_target = 0.4
batch = 1
horizon = map_freq(freq)
weights_bandwidth = 0.01
universe = Universe('zz800')
data_source = os.environ['DB_URI']
benchmark_code = 905
method = 'risk_neutral'

In [ ]:
# Model settings
alpha_factors = {
    'f01': CSQuantiles(LAST('EPS'), groups='sw1'),
    'f02': CSQuantiles(LAST('ROE'), groups='sw1'),
    }

weights = dict(f01=1.,
               f02=1.)

alpha_model = ConstLinearModel(features=alpha_factors, weights=weights)

data_meta = DataMeta(freq=freq,
                     universe=universe,
                     batch=1,
                     neutralized_risk=None,
                     pre_process=None,
                     post_process=None,
                     data_source=data_source)

In [ ]:
# Constraintes settings

industry_names = industry_list(industry_name, industry_level)
constraint_risk = ['SIZE', 'SIZENL', 'BETA']
total_risk_names = constraint_risk + ['benchmark', 'total']
all_styles = risk_styles + industry_styles + macro_styles

b_type = []
l_val = []
u_val = []

previous_pos = pd.DataFrame()
rets = []
turn_overs = []
leverags = []

for name in total_risk_names:
    if name == 'benchmark':
        b_type.append(BoundaryType.RELATIVE)
        l_val.append(0.8)
        u_val.append(1.0)
    else:
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(0.0)
        u_val.append(0.0)

bounds = create_box_bounds(total_risk_names, b_type, l_val, u_val)

In [ ]:
# Running settings
running_setting = RunningSetting(universe,
                                 start_date,
                                 end_date,
                                 freq,
                                 benchmark=benchmark_code,
                                 weights_bandwidth=weights_bandwidth,
                                 rebalance_method=method,
                                 bounds=bounds,
                                 turn_over_target=turn_over_target)

In [ ]:
# Strategy
strategy = Strategy(alpha_model, data_meta, running_setting)
ret_df, positions = strategy.run()

In [ ]:
ret_df[['turn_over', 'excess_return']].cumsum().plot(figsize=(14, 7), secondary_y='turn_over')